In [ ]:
pip install selenium
pip install webdriver-manager
pip install pandas

# 파워쉘에서 설치
# 주피터노트북에서 설치 시 앞에 ! 붙이기
# pip install selenium webdriver-manager pandas 형태로 한줄로 설치해도 된다. 이미 설치되어있다면 Requirement already satisfied:라고 뜬다.

In [23]:
import time
import pandas as pd

# Selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# webdriver 설정하기
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

# 인스타그램 열기
driver.get("https://www.instagram.com/")

In [24]:
# 나의 ID와 PW를 이용해 로그인 진행
myID = input("인스타그램 ID를 입력하세요: ")
myPW = input("인스타그램 PW를 입력하세요: ")

# Class name을 이용해 로그인 ID, PW 입력 칸 특정하기
from selenium.webdriver.common.by import By

instagramID, instagramPW = driver.find_elements(By.CLASS_NAME, "_aa48")

# ID칸을 누르고 내 ID입력
instagramID.click()
instagramID.send_keys(myID)

# PW칸을 누르고 내 PW입력
instagramPW.click()
instagramPW.send_keys(myPW)

# 로그인 버튼 누르기
driver.find_element(By.XPATH,'//*[@id="loginForm"]/div[1]/div[3]/button/div').click()

In [ ]:
import re
import time
import unicodedata
from bs4 import BeautifulSoup

# 게시글 본문 + 해시태그 추출 함수
def get_content(driver):
    time.sleep(2) 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # <meta name="description">에서 본문 추출
    try:
        desc = soup.select_one('meta[name="description"]')['content']
        content = desc.split(':', 1)[-1].strip().strip('"').replace('\n', ' ')
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''

    # 해시태그 추출
    tags = re.findall(r'#[^\s#,\\]+', content)

    return content, tags

# 크롤링 실행
results = []

# 사용자 입력
category = input("구분 업종: ")
keyword = input("어떤 키워드로 검색?: ")
search_url = f"https://www.instagram.com/explore/tags/{keyword}/?next=%2F"

driver.get(search_url)
time.sleep(5)

# 설정값
TARGET_TOTAL = 5
TARGET_REEL = 1
MAX_SCROLL = 20

post_links = []
scroll_count = 0
reel_count = 0
feed_count = 0

# 게시물 링크 수집
while True:
    links = driver.find_elements(By.CSS_SELECTOR, "a[href^='/']")
    for link in links:
        href = link.get_attribute("href")
        if href and ("/p/" in href or "/reel/" in href) and href not in post_links:
            post_links.append(href)

    print(f"[스크롤 {scroll_count+1}] 링크 수집: {len(post_links)}개")

    if len(post_links) >= TARGET_TOTAL * 2:
        break

    scroll_count += 1
    if scroll_count >= MAX_SCROLL:
        break

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 게시물 상세 정보 수집
for url in post_links:
    if len(results) >= TARGET_TOTAL and reel_count >= TARGET_REEL:
        break

    driver.get(url)
    time.sleep(2)

    # 릴스 여부
    try:
        driver.find_element(By.TAG_NAME, "video")
        channel_type = "ig_reels"
        reel_count += 1
    except:
        channel_type = "ig_feed"
        feed_count += 1

    # 본문 및 해시태그
    caption, hashtags = get_content(driver)

    results.append({
        "채널/기법": channel_type,
        "구분업종": category,
        "검색 키워드": keyword,
        "게시물 링크": url,
        "게시물 텍스트": caption,
        "해시태그 리스트": " ".join(hashtags)
    })

print(f"\n수집 완료: 릴스 {reel_count}개 / 피드 {feed_count}개")


[스크롤 1] 링크 수집: 18개

수집 완료: 릴스 1개 / 피드 4개


In [29]:
from datetime import datetime

# 현재 시간 문자열 (2025-07-29_09-30-15)
now_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 파일명 생성
filename = f"ig_crawling_result_{now_str}.csv"

# 저장
df = pd.DataFrame(results)
df.to_csv(filename, index=False, encoding="utf-8-sig")
print(f"CSV 저장 완료: {filename}")

CSV 저장 완료: ig_crawling_result_2025-07-29_10-37-21.csv
